# LLM Model

In [3]:
# LLM Model
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

# Change LCEL
prompt = PromptTemplate.from_template("what is the famous tour place in {city}?")
# chain = LLMChain(llm=model, prompt=prompt)
city = "Seoul"
# chain.run(city)
chain = prompt | model
chain.invoke({"city": city})

AIMessage(content="Seoul is home to many famous tourist attractions. Some of the most popular places to visit include:\n\n1. **Gyeongbokgung Palace**: The largest of the Five Grand Palaces built during the Joseon Dynasty, known for its beautiful architecture and the changing of the guard ceremony.\n\n2. **Bukchon Hanok Village**: A traditional Korean village featuring hanok (traditional Korean houses) that offers a glimpse into Korea's past.\n\n3. **N Seoul Tower (Namsan Tower)**: An iconic landmark that provides panoramic views of the city, especially beautiful at night.\n\n4. **Myeongdong**: A bustling shopping district known for its street food, cosmetics shops, and fashion boutiques.\n\n5. **Insadong**: A cultural district famous for its art galleries, traditional tea houses, and shops selling Korean crafts and antiques.\n\n6. **Dongdaemun Design Plaza (DDP)**: A modern architectural marvel that hosts exhibitions, fashion shows, and other cultural events.\n\n7. **Changdeokgung Pala

# Sequential Chain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template(
    "what is the famous tour place in {city}? Tell me the name of the place only without additional comments."
)
prompt2 = PromptTemplate.from_template("How can I get {place} by {transport}?")

# Old version
# chain1 = LLMChain(llm=model, prompt=prompt1, output_key="place", verbose=True)
# chain2 = LLMChain(llm=model, prompt=prompt2, verbose=True)
# chain = SequentialChain(
#     chains=[chain1, chain2], input_variables=["city", "transport"], verbose=True
# )
# chain.run({"city": "Seoul", "transport": "subway"})

# LCEL
chain_1 = prompt1 | model | StrOutputParser()
chain_2 = prompt2 | model
sequential_chain = (
    {
        "place": RunnablePassthrough(),
        "transport": RunnablePassthrough(),
        "city": RunnablePassthrough(),
    }
    | chain_1
    | {
        "place": RunnablePassthrough(),
        "transport": RunnablePassthrough(),
    }
    | chain_2
)
sequential_chain.invoke({"city": "Seoul", "transport": "subway"})


AIMessage(content='It seems like you might be asking how to get to Gyeongbokgung Palace in Seoul, South Korea. If you are already at Gyeongbokgung Palace, you may be looking for directions to a specific location within or nearby the palace. \n\nIf you are looking for directions to Gyeongbokgung Palace from another location, here are some general options:\n\n1. **Subway**: The easiest way to get to Gyeongbokgung Palace is by taking the Seoul Subway. You can take Line 3 (the Orange Line) and get off at Gyeongbokgung Station. Use Exit 5, and the palace will be a short walk from there.\n\n2. **Bus**: Several city buses stop near Gyeongbokgung Palace. You can check local bus routes for the best option based on your starting point.\n\n3. **Taxi**: You can also take a taxi to Gyeongbokgung Palace. Just tell the driver "Gyeongbokgung" (경복궁), and they will know where to take you.\n\n4. **Walking**: If you are nearby, you can simply walk to the palace. It is located in the heart of Seoul and is 

# Sequential Advacned

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template(
    "what is the famous tour place in {city}? Tell me the name of the place only without additional comments."
)
prompt2 = PromptTemplate.from_template(
    "What is the top 5 restaurant in the {place} in city {city} without additional comments?"
)  # output : restaurants
prompt3 = PromptTemplate.from_template(
    "What is the best one restaurant and food for family dinner among {restaurants} ?"
)  # output : restaurant_information
prompt4 = PromptTemplate.from_template(
    "How can I get the {place} by using {transport}?"
)  # output : transport_information
final_prompt = PromptTemplate.from_template(
    """
Please summarize the tour information with reastaurant information and transportation by using the this information.
Restaurant informations : {restaurant_information}
Transport information : {transport_information}
"""
)

# Old version
# chain1 = LLMChain(llm=model, prompt=prompt1, output_key="place", verbose=True)
# chain2 = LLMChain(llm=model, prompt=prompt2, output_key="restaurants", verbose=True)
# chain3 = LLMChain(
#     llm=model, prompt=prompt3, output_key="restaurant_information", verbose=True
# )
# chain4 = LLMChain(
#     llm=model, prompt=prompt4, output_key="transport_information", verbose=True
# )
# final_chain = LLMChain(
#     llm=model, prompt=final_prompt, output_key="tour_summary", verbose=True
# )
# chain = SequentialChain(
#     chains=[chain1, chain2, chain3, chain4, final_chain],
#     input_variables=["city", "transport"],
#     verbose=True,
# )
# chain.run({"city": "Seoul", "transport": "subway"})

# New LCEL
chain_1 = prompt1 | model | StrOutputParser()
chain_2 = prompt2 | model | StrOutputParser()
chain_3 = prompt3 | model | StrOutputParser()
chain_4 = prompt4 | model | StrOutputParser()
sequential_chain = (
    {
        "transport": RunnablePassthrough(),
        "city": RunnablePassthrough(),
    }
    | chain_1
    | {
        "city": RunnablePassthrough(),
        "place": RunnablePassthrough(),
        "transport": RunnablePassthrough(),
    }
    | chain_2
    | {
        "restaurants": RunnablePassthrough(),
        "place": RunnablePassthrough(),
        "transport": RunnablePassthrough(),
    }
    | chain_3
    | {
        "restaurant_information": RunnablePassthrough(),
        "place": RunnablePassthrough(),
        "transport": RunnablePassthrough(),
    }
    | chain_4
    | {
        "transport_information": RunnablePassthrough(),
        "restaurant_information": RunnablePassthrough(),
    }
    | final_prompt
    | model
)
sequential_chain.invoke({"city": "Seoul", "transport": "subway"})

AIMessage(content="### Tour and Dining Summary\n\n**Restaurant Options:**\n\n1. **Tosokchon Samgyetang**\n   - **Cuisine**: Traditional Korean (famous for ginseng chicken soup)\n   - **Atmosphere**: Cozy and authentic\n   - **Dining Experience**: More formal setting\n   - **Recommendation**: Ideal for a traditional Korean experience.\n\n2. **Hanok Jib**\n   - **Cuisine**: Variety of Korean dishes\n   - **Atmosphere**: Unique traditional setting in a hanok\n   - **Dining Experience**: More formal\n   - **Recommendation**: Great for those seeking a traditional dining experience.\n\n3. **Bukchon Son Mandu**\n   - **Cuisine**: Comfort food (specializes in handmade dumplings)\n   - **Atmosphere**: Casual and relaxed\n   - **Dining Experience**: Laid-back\n   - **Recommendation**: Perfect for families, especially with kids.\n\n4. **The Lounge & Bar at The Four Seasons Hotel Seoul**\n   - **Cuisine**: International and upscale\n   - **Atmosphere**: Refined and luxurious\n   - **Dining Experie

# Router Chain

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.output_parsers import StrOutputParser

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

prompt_infos = [
    {
        "name": "restaurants",
        "description": "Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template,
    },
    {
        "name": "transport",
        "description": "Good for guiding the transport to get the place",
        "prompt_template": transport_template,
    },
    {
        "name": "destination",
        "description": "Good for recommending place to tour",
        "prompt_template": destination_template,
    },
]

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

router_prompt = PromptTemplate.from_template(
    """
Given the following options, choose the one that best fits with the input.
Destinations:
{destinations}

Input: {input}

Return the name of the destination only.
"""
)

router_chain = {"input": RunnablePassthrough(), "destinations": RunnableLambda(lambda x: destinations_str)} | router_prompt | model | StrOutputParser()


destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt_template = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = prompt_template | model | StrOutputParser()
    destination_chains[name] = chain


branch = RunnableBranch(
    *[
        (
            lambda x, name=p["name"]: x["destination"].lower() == name.lower(),
            destination_chains[p["name"]]
        )
        for p in prompt_infos
    ],
    destination_chains[prompt_infos[0]["name"]]
)


multi_prompt_chain = (
    {
        "input": RunnablePassthrough(),
        "destination": router_chain
    }
    | branch
)

multi_prompt_chain.invoke("What is the best restaurant in Seoul?")

'Seoul is a vibrant city with a diverse culinary scene, making it hard to pinpoint just one "best" restaurant. However, one highly recommended spot is **Mingles**, which has received acclaim for its innovative take on traditional Korean cuisine. The chef, Mingoo Kang, blends seasonal ingredients with modern techniques, creating a unique dining experience.\n\nIf you\'re looking for something more traditional, **Gwangjang Market** is a must-visit. Here, you can sample a variety of authentic Korean street foods, such as bindaetteok (mung bean pancakes) and bibimbap.\n\nFor a fine dining experience, consider **Jungsik**, which offers a contemporary approach to Korean dishes and has earned two Michelin stars.\n\nIf you\'re in the mood for Korean barbecue, **Maple Tree House** is a popular choice, known for its high-quality meats and cozy atmosphere.\n\nEach of these places offers a different taste of Seoul\'s culinary landscape, so it really depends on what kind of dining experience you\'re